# Forms/Instruments {#sec-forms_instruments}

::: {.content-hidden when-format="html"}

## Project Setup

Install and load the necessary packages

In [3]:
#| echo: false
#| output: false
import os
os.environ['R_HOME'] = f'C:/Users/{os.environ.get('USERNAME')}/Miniconda3/envs/r_python_jl/Lib/R'

In [4]:
#| echo: false
#| output: false
%load_ext rpy2.ipython
# only have to run once to allow the R magic command

C:\Users\ekp0303\Miniconda3\envs\r_python_jl\Lib\site-packages\rpy2\robjects\packages.py:367: UserWarning: The symbol 'quartz' is not in this R namespace/package.
  warnings.warn(


::: {.panel-tabset}

#### R

In [5]:
%%capture 
%%R

library("dplyr")
library("jsonlite")
library("tidyr")
library("REDCapR")
library("knitr")
library("remotes")
library("gt")

In [6]:
%%capture --no-display --no-stdout
%%R

version <- packageVersion("REDCapR")
version

[1] '1.1.9005'


In this project, we will use the bleeding edge version of REDCapR available on Github

In [7]:
%%capture --no-display --no-stdout
%%R

# Detach REDCapR if already loaded, and download the latest version
if (version!='1.1.9005') {
    detach("package:REDCapR", unload=TRUE)
    remotes::install_github("OuhscBbmc/REDCapR")
    library("REDCapR")
    print(packageVersion("REDCapR"))
}

else {
    print("REDCapR package up to date")
}

[1] "REDCapR package up to date"


#### Python

In [8]:
import redcap
import json
import pandas as pd

:::

Assign your project URL and Token

::: {.panel-tabset}

#### R

In [9]:
%%R
path = paste0("C:/Users/", Sys.getenv("USERNAME"), '/json_api_data.json')
token <- jsonlite::fromJSON(path)$dev_token$'308'
url <- "https://dev-redcap.doh.wa.gov/api/"

#### Python

In [10]:
path_to_json = f"C:/Users/{os.environ.get('USERNAME')}/json_api_data.json"
api_key = json.load(open(path_to_json))
api_token = api_key['dev_token']['308']
api_url = api_key['dev_url']
project = redcap.Project(api_url, api_token)

:::

:::

## Instrument Names and Labels

You can export a list of instrument names and their corresponding instrument labels. 

::: {.panel-tabset}

#### R

`redcap_instruments()`

In [11]:
%%capture
%%R
instruments <- redcap_instruments(
    redcap_uri = url, 
    token = token, 
    verbose = TRUE, 
    config_options = NULL
)$data

::: {.content-hidden when-format="html"}

In [2]:
%%R
instruments_tbl <- gt(head(instruments))
gt::gtsave(instruments_tbl, filename = 'instruments.html', path = "./files/export_files/")

UsageError: Cell magic `%%R` not found.


:::

<p align="center"><iframe width="100%" height="270" src="./files/export_files/instruments.html" title="Quarto Documentation"></iframe></p>

#### Python

`export_instruments()`

In [11]:
project.export_instruments(format_type = "df")

,instrument_name,instrument_label
0,demographics,Demographics
1,symptoms,Symptoms
2,test_information,Test Information
3,close_contacts,Close Contacts
4,work_information,Work Information


:::

**Note:** Use the value under `instrument_name` (not `instrument_label`) when specifying specific instruments as arguments in the REDCapR and PyCap API functions.

## Download PDF of Instruments

These functions will download the instrument specified and all of the fields as a cleanly formatted questionnaire pdf file. 

::: {.panel-tabset}

#### R

`redcap_instrument_download()`

In [12]:
%%capture
%%R
redcap_instrument_download(
    instrument = "symptoms",
    directory = "./files/instruments/",
    file_name = "symptoms_instrument.pdf",
    redcap_uri = url,
    overwrite = TRUE,
    token = token
)

<p align="center"><iframe width="100%" height="500" src="./files/instruments/symptoms_instrument.pdf" title="Quarto Documentation"></iframe></p>

#### Python

`export_pdf()`

In [13]:
close_contact = project.export_pdf(instrument = 'close_contacts')
with open("files/instruments/close_contacts.pdf","wb") as binary_file:
    binary_file.write(close_contact[0])

In [18]:
from IPython.display import IFrame
IFrame("files/instruments/close_contacts.pdf", width=750, height=500)

:::